In [ ]:
# Packages needed
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
import en_core_web_sm
import matplotlib.pyplot as plt
import matplotlib
plt.style.use('fivethirtyeight')

In [ ]:
nlp = en_core_web_sm.load()

In [ ]:
train_data=pd.read_csv('train_script3.csv')
test_data = pd.read_csv('test_script3.csv')

In [ ]:
data = pd.concat([train_data, test_data]).reset_index()

data= data.dropna().reset_index()
data = data[['lines', 'character']]

### Tokenize

In [ ]:
from nltk.tokenize import word_tokenize
def get_tokens(text):
    return word_tokenize(text)

## Emotion

In [ ]:
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [ ]:
# Download the emotion lexicon

filepath = ('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
emolex_df = pd.read_csv(filepath, names=["word", "emotion", "association"], sep='\t')

In [ ]:
emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
emotions = emolex_words.columns.drop('word')
emo_df = pd.DataFrame(0, index=data.index, columns=emotions)

In [ ]:
data = pd.concat([data, emo_df], axis=1)

In [ ]:
# Create a 
def text_emotion(df):


    new_df = df.copy()

    filepath = ('NRC-Emotion-Lexicon-Wordlevel-v0.92.txt')
    emolex_df = pd.read_csv(filepath, names=["word", "emotion", "association"], sep='\t')
    emolex_words = emolex_df.pivot(index='word', columns='emotion', values='association').reset_index()
    emotions = emolex_words.columns.drop('word')
    emo_df = pd.DataFrame(0, index=df.index, columns=emotions)

    stemmer = SnowballStemmer("english")
    
    for i in range(len(df)):
        document = new_df.lines[i]
        document = word_tokenize(df.loc[i]['lines'])
        print(i)
        
        for word in document:
            #word = stemmer.stem(word.lower()) Stemmer, there is also an additional example of using the stemming 
            emo_score = emolex_words[emolex_words.word == word]
            if not emo_score.empty:
                for emotion in list(emotions):
                    emo_df.at[i, emotion] += emo_score[emotion]

    new_df = pd.concat([new_df, emo_df], axis=1)

    return new_df

In [ ]:
emotion_data = text_emotion(data)

In [ ]:
emotion_data['word_count'] = emotion_data['lines'].apply(word_tokenize).apply(len)

In [ ]:
emotions = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust']

In [ ]:
emotion_data2 = pd.DataFrame(columns =['character', 'anger','anticipation', 'disgust', 'fear', 'joy', 'negative', 'positive', 'sadness', 'surprise', 'trust', 'word_count'])

In [ ]:
emotion_data2['character'] =['Leonard','Sheldon','Penny', 'Howard','Raj','Amy','Bernadette']
emotion_data2

In [ ]:
for character in ['Leonard','Sheldon','Penny', 'Howard','Raj','Amy','Bernadette']:
    print(character)
    columns = emotions + ['word_count']
    A = emotion_data2.index[emotion_data2['character'] == character]
    idx = A[0]
    for emotion in columns:
        emotion_data2[emotion][idx] = emotion_data.loc[emotion_data['character'] == character, emotion].sum()
 

In [ ]:
emotion_data2

In [ ]:
emotion_data3 = emotion_data2.copy(deep=True)

In [ ]:
emotion_data3

In [ ]:
for emotion in emotions:
    emotion_data3[emotion] = emotion_data3[emotion] / emotion_data3['word_count']

In [ ]:
for emotion in emotions:
    for i in range(7):
        emotion_data3[emotion][i] = emotion_data3[emotion][i].tolist()[1]

In [ ]:
emotion_data3

In [ ]:
emotion_data3.to_csv('emotions_no_stemming.csv')

In [ ]:
emotions